# Oracle Eloqua Testing

### Setup

In [11]:
%autosave 600

import os
import sys

Autosaving every 600 seconds


In [12]:
# stupid and lazy way to set up location; 
# doesn't matter since this file will go bye-bye
proxies = {}
if os.name == 'nt':
    prefix = 'H:'
    proxies['http'] = proxies['https'] = 'wsa04proxy.int.capella.lan:80'
else:
    prefix = '/Users/nickolasgryga'

sys.path.append(prefix + '/work/oracle_eloqua')
creds_loc = prefix + '/work/adhoc/credentials.json'

### Test Import

In [3]:
import oracle_eloqua

### Test API Connection

In [16]:
from oracle_eloqua import EloquaApi
api = oracle_eloqua.EloquaApi.from_json(creds_loc, proxies=proxies)
print(api)

### Test Cursor

In [17]:
from oracle_eloqua import Cursor
request = Cursor(
    path=('assets','campaigns'),
    api_type='rest',
    params={},
    api=api
)

In [18]:
response = request.fetchall()

In [19]:
len(response['elements'])

10746

### Test Requests